# Noticed that a lot of the addresses don't have the postcode in them

- This is bad as I would like to have a column of postcode outcodes
- Hence I am going to follow this tutorial on how to get Zip Code with given location using GeoPy in Python
- I also used this file to play around with designing functions such as the find_postcode_in_address() function

https://www.geeksforgeeks.org/get-zip-code-with-given-location-using-geopy-in-python/

In [192]:
# pip install geopy

In [206]:
# import module
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re

In [200]:
# initialize Nominatim API
geolocator = Nominatim(user_agent="myGeolocator", timeout=None)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.0, error_wait_seconds=2.0, swallow_exceptions=True, return_value_on_exception=None)

In [201]:
place = "South Street, Normanton, WF6"
location = geolocator.geocode(place)

In [202]:
location

Location(South Street, Normanton, Warmfield, Wakefield, West Yorkshire, England, WF6 1EE, United Kingdom, (53.6879563, -1.4230688, 0.0))

In [203]:
data = location.raw
print(data)

{'place_id': 115009476, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 36892995, 'boundingbox': ['53.6875729', '53.6879619', '-1.4235386', '-1.4224892'], 'lat': '53.6879563', 'lon': '-1.4230688', 'display_name': 'South Street, Normanton, Warmfield, Wakefield, West Yorkshire, England, WF6 1EE, United Kingdom', 'class': 'highway', 'type': 'residential', 'importance': 0.51}


In [204]:
loc_data = data['display_name'].split()
print("Full Location")
print(loc_data)
print("Out code : ",loc_data[-4])

Full Location
['South', 'Street,', 'Normanton,', 'Warmfield,', 'Wakefield,', 'West', 'Yorkshire,', 'England,', 'WF6', '1EE,', 'United', 'Kingdom']
Out code :  WF6


In [251]:
import pandas as pd
propertiescopy = pd.read_csv('../Data/properties_geocoded.csv')

In [252]:
propertiescopy

,url,address,property_type,bedrooms,bathrooms,tenure,nearest_station,price,cleaned_address,gcode
0,https://www.rightmove.co.uk/properties/1315844...,"Queen Square, Leeds",Detached,5,2,Freehold,0.7,550000,"Queen Square, Leeds","Queen Square, Arena Quarter, Lovell Park, Leed..."
1,https://www.rightmove.co.uk/properties/8604795...,"Archery Road, Leeds, West Yorkshire",Terraced,5,1,Freehold,1.0,325000,"Archery Road, Leeds, West Yorkshire","Archery Road, Arena Quarter, Woodhouse, Leeds,..."
2,https://www.rightmove.co.uk/properties/1326138...,"Hanover Square, Leeds",Terraced,6,3,Freehold,0.8,450000,"Hanover Square, Leeds","Hanover Square, Hyde Park, Leeds, West Yorkshi..."
3,https://www.rightmove.co.uk/properties/1311469...,"Burley Wood Crescent, Burley, LEEDS",Terraced,3,1,Freehold,0.5,220000,"Burley Wood Crescent, Burley, LEEDS","Burley Wood Crescent, Burley, Leeds, West York..."
4,https://www.rightmove.co.uk/properties/1322668...,"The Village Street, Burley, Leeds",Detached,5,2,Freehold,0.2,325000,"The Village Street, Burley, Leeds","The Village Street, Burley, Leeds, West Yorksh..."
...,...,...,...,...,...,...,...,...,...,...
4557,https://www.rightmove.co.uk/properties/1328121...,"Royds Avenue, New Mill, Holmfirth, West Yorksh...",Terraced,3,1,Freehold,1.5,160000,"Royds Avenue, New Mill, Holmfirth, West Yorksh...",NaN
4558,https://www.rightmove.co.uk/properties/1328843...,"Greenhill Bank Road, New Mill, Holmfirth, West...",Terraced,2,1,Freehold,1.6,150000,"Greenhill Bank Road, New Mill, Holmfirth, West...",NaN
4559,https://www.rightmove.co.uk/properties/1281187...,"Thick Hollins, Meltham, Holmfirth, West Yorksh...",Terraced,2,1,Freehold,2.8,150000,"Thick Hollins, Meltham, Holmfirth, West Yorksh...",NaN
4560,https://www.rightmove.co.uk/properties/1328625...,"Holmclose, Holmbridge, Holmfirth, West Yorkshi...",Semi-Detached,2,1,Freehold,3.4,150000,"Holmclose, Holmbridge, Holmfirth, West Yorkshi...",NaN


In [253]:
def find_postcodes_in_address(data):
    """
    A function that finds addresses that do have a postcode but the outcode column hasn't got a value for this due to geopcode method not being able to find it
    :param data: A dataframe to apply the find_numerical_answers() function to
    :return: A dataframe with a numerical_answer column and value for each row
    """
    data['outcode_from_address'] = data.apply(match_postcodes_in_address, axis=1)
    return data

In [254]:
def match_postcodes_in_address(row):
    """
    Function to see if the cleaned address column for rows that don't have an outcode has the outcode
    Then return that outcode and assign it to the row
    :param row: A row of a dataframe
    :return 1 or NaN: Returns a value of 1 to denote if an answer is numerical in nature
    """
    address = row['cleaned_address']

    # Regexes for outcodes for leeds, huddersfield
    outcode_regexes = [r'LS\d{1,2}', r'HD\d{1,2}', r'HX\d{1,2}', r'WF\d{1,2}', r'BD\d{1,2}']

    ldsmatch = re.findall(outcode_regexes[0], address)
    hudsmatch = re.findall(outcode_regexes[1], address)
    hfxmatch = re.findall(outcode_regexes[2], address)
    wfmatch = re.findall(outcode_regexes[3], address)
    bdmatch = re.findall(outcode_regexes[4], address)

    if ldsmatch:
        return ldsmatch[0]
    elif hudsmatch:
        return hudsmatch[0]
    elif hfxmatch:
        return  hfxmatch[0]
    elif wfmatch:
        return wfmatch[0]
    elif bdmatch:
        return bdmatch[0]
    else:
        return None

In [255]:
# Let's see if I can reduce the number of null outcodes
propertiescopy = find_postcodes_in_address(propertiescopy)

In [256]:
propertiescopy[propertiescopy['outcode_from_address'].notna()]

,url,address,property_type,bedrooms,bathrooms,tenure,nearest_station,price,cleaned_address,gcode,outcode_from_address
8,https://www.rightmove.co.uk/properties/1285506...,"Lumley Avenue, Leeds, West Yorkshire, LS4",Terraced,4,2,Freehold,0.1,250000,"Lumley Avenue, Leeds, West Yorkshire, LS4","Lumley Avenue, Burley, Leeds, West Yorkshire, ...",LS4
11,https://www.rightmove.co.uk/properties/1327622...,"Kirkstall Road, Leeds, LS4 2QD",Semi-Detached,3,1,Freehold,0.6,235000,"Kirkstall Road, Leeds, LS4 2QD","Kirkstall Road, Burley, Leeds, West Yorkshire,...",LS4
12,https://www.rightmove.co.uk/properties/1317795...,"Burley Wood Crescent, Leeds, LS4 2QQ",Terraced,3,1,Freehold,0.5,230000,"Burley Wood Crescent, Leeds, LS4 2QQ","Burley Wood Crescent, Burley, Leeds, West York...",LS4
17,https://www.rightmove.co.uk/properties/1328079...,"Haddon Avenue, Leeds, LS4",Terraced,2,1,Freehold,0.4,180000,"Haddon Avenue, Leeds, LS4","Haddon Avenue, Burley, Leeds, West Yorkshire, ...",LS4
19,https://www.rightmove.co.uk/properties/1311390...,"Morris Grove, Leeds, LS5",Terraced,2,1,Freehold,0.2,240000,"Morris Grove, Leeds, LS5","Morris Grove, Kirkstall, Leeds, West Yorkshire...",LS5
...,...,...,...,...,...,...,...,...,...,...,...
4557,https://www.rightmove.co.uk/properties/1328121...,"Royds Avenue, New Mill, Holmfirth, West Yorksh...",Terraced,3,1,Freehold,1.5,160000,"Royds Avenue, New Mill, Holmfirth, West Yorksh...",NaN,HD9
4558,https://www.rightmove.co.uk/properties/1328843...,"Greenhill Bank Road, New Mill, Holmfirth, West...",Terraced,2,1,Freehold,1.6,150000,"Greenhill Bank Road, New Mill, Holmfirth, West...",NaN,HD9
4559,https://www.rightmove.co.uk/properties/1281187...,"Thick Hollins, Meltham, Holmfirth, West Yorksh...",Terraced,2,1,Freehold,2.8,150000,"Thick Hollins, Meltham, Holmfirth, West Yorksh...",NaN,HD9
4560,https://www.rightmove.co.uk/properties/1328625...,"Holmclose, Holmbridge, Holmfirth, West Yorkshi...",Semi-Detached,2,1,Freehold,3.4,150000,"Holmclose, Holmbridge, Holmfirth, West Yorkshi...",NaN,HD9
